In [1]:
import ephem

precess_pos = ephem.FixedBody()
precess_pos._ra = '10:35:57.83'
precess_pos._dec = -0.536191810965
precess_pos._epoch = '2016/6/8 16:00:23'

print precess_pos._ra, precess_pos._dec
print '10:45:57.83', precess_pos._dec

precess_pos.compute(epoch=ephem.J2000)
print 'J2000'
print precess_pos.a_ra, precess_pos.a_dec

del(precess_pos)

#now increase ra and epoch by 10 minutes
precess_pos = ephem.FixedBody()
precess_pos._ra = '10:45:57.83'
precess_pos._dec = -0.536191810965
precess_pos._epoch = '2016/6/8 16:10:23'

precess_pos.compute(epoch=ephem.J2000)
print precess_pos.a_ra, precess_pos.a_dec


10:35:57.83 -30:43:17.5
10:45:57.83 -30:43:17.5
J2000
10:35:11.98 -30:38:10.2
10:45:11.44 -30:38:05.3


In [2]:
import astropy.units as units
from astropy.coordinates import EarthLocation, SkyCoord, FK5, Angle
fk5c_1 = SkyCoord('10h35m57.83s', Angle(-0.536191810965, unit='radian'), frame=FK5, equinox='2016-6-8T16:00:23')
fk5_2000 = FK5(equinox='J2000')
precess_pos1 = fk5c_1.transform_to(fk5_2000)
precess_pos1_icrs = fk5c_1.transform_to('icrs')

fk5c_2 = SkyCoord('10h45m57.83s', Angle(-0.536191810965, unit='radian'), frame=FK5, equinox='2016-6-8T16:10:23')
precess_pos2 = fk5c_2.transform_to(fk5_2000)
precess_pos2_icrs = fk5c_2.transform_to('icrs')

print fk5c_1.ra.to_string(unit='hourangle', sep='hms'), fk5c_1.dec
print fk5c_2.ra.to_string(unit='hourangle', sep='hms'), fk5c_2.dec
print 'FK5 J2000'
print precess_pos1.ra.to_string(unit='hourangle', sep='hms'), precess_pos1.dec
print precess_pos2.ra.to_string(unit='hourangle', sep='hms'), precess_pos2.dec
print 'icrs'
print precess_pos1_icrs.ra.to_string(unit='hourangle', sep='hms'), precess_pos1_icrs.dec
print precess_pos2_icrs.ra.to_string(unit='hourangle', sep='hms'), precess_pos2_icrs.dec


10h35m57.83s -30d43m17.5s
10h45m57.83s -30d43m17.5s
FK5 J2000
10h35m11.9819s -30d38m10.1998s
10h45m11.4474s -30d38m05.3207s
icrs
10h35m11.9798s -30d38m10.1841s
10h45m11.4452s -30d38m05.3057s


Now try to figure out how to get "current RA/Dec" with astropy

In [13]:
from astropy.time import Time
obs_time = '2016-6-8T16:00:23'
array_location = EarthLocation(lat='-30d43m17.5s', lon='21d25m41.9s',
                               height=1073.)

source_coord = SkyCoord(alt=Angle(90 * units.deg), az=Angle(0 * units.deg),
                        obstime=obs_time, frame='altaz', location=array_location)
icrs_coord = source_coord.transform_to('icrs')

ra_icrs = icrs_coord.ra
dec_icrs = icrs_coord.dec

t_obj = Time(obs_time, scale='utc', location=(array_location.lon, array_location.lat))
lst = t_obj.sidereal_time('apparent')
print lst.to_string(unit='hourangle', sep='hms'), array_location.lat

print 'icrs:'
print ra_icrs.to_string(unit='hourangle', sep='hms'), dec_icrs

obs_coord = source_coord.transform_to(FK5(equinox=obs_time))
ra_obs = obs_coord.ra
dec_obs = obs_coord.dec
print 'obs:'
print ra_obs.to_string(unit='hourangle', sep='hms'), dec_obs


10h35m57.2539s -30d43m17.5s
icrs:
10h35m11.5052s -30d37m56.8212s
obs:
10h35m57.3557s -30d43m04.133s


In [28]:
from astropy.time import TimeDelta
t1 = Time('2016-6-8T16:00:23', scale='utc', location=(array_location.lon, array_location.lat))
t2 = Time('2016-6-9T16:00:23', scale='utc', location=(array_location.lon, array_location.lat))
delta_lst = t2.sidereal_time('apparent') - t1.sidereal_time('apparent') 
delta_lst.rad

0.017202885878270565

In [33]:
t1 = Time('2016-6-8T16:00:23', scale='utc', location=(array_location.lon, array_location.lat))
dt = TimeDelta(2.0, format='sec')
t2 = t1 + dt
print(t2)
delta_lst = t2.sidereal_time('apparent') - t1.sidereal_time('apparent') 
delta_lst.rad

2016-06-08T16:00:25.000


0.00014584232149872391